In [2]:
!pip install diffusers transformers accelerate bitsandbytes safetensors datasets wandb --upgrade
!pip install opencv-python pillow


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Change to your dataset path
data_dir = "/content/drive/MyDrive/archive (11)"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import pandas as pd

# Read all your data splits
train_df = pd.read_csv(os.path.join(data_dir, "/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata/train_data.csv"))
val_df = pd.read_csv(os.path.join(data_dir, "/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata/val_data.csv"))
meta_df = pd.read_csv(os.path.join(data_dir, "/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata/metadata.csv"))

print(train_df.head())
print(val_df.head())
print(meta_df.head())


                                          image_path    room_type  \
0  ../data/raw/kitchen/modern/kitchen_modern_224.jpg      kitchen   
1  ../data/raw/living_room/boho/living_room_boho_...  living_room   
2  ../data/raw/kitchen/industrial/kitchen_industr...      kitchen   
3  ../data/raw/bedroom/minimalist/bedroom_minimal...      bedroom   
4  ../data/raw/bathroom/scandinavian/bathroom_sca...     bathroom   

          style  encoded_label  
0        modern             13  
1          boho             15  
2    industrial             11  
3    minimalist              7  
4  scandinavian              4  
                                          image_path room_type       style  \
0      ../data/raw/kitchen/boho/kitchen_boho_219.jpg   kitchen        boho   
1  ../data/raw/bathroom/industrial/bathroom_indus...  bathroom  industrial   
2     ../data/raw/bathroom/boho/bathroom_boho_83.jpg  bathroom        boho   
3  ../data/raw/bathroom/industrial/bathroom_indus...  bathroom  industrial 

In [5]:
import os
import pandas as pd

base_dir = "/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata"

# List your folders and a few image examples
for folder in ["bathroom", "bedroom", "kitchen", "living_room"]:
    path = os.path.join(base_dir, folder)
    if os.path.exists(path):
        print(f"\n📁 {folder} → {len(os.listdir(path))} files")
        print("Example files:", os.listdir(path)[:5])
    else:
        print(f"❌ Missing folder: {folder}")

# Preview metadata
meta = pd.read_csv(os.path.join(base_dir, "metadata.csv"))
print("\n🧾 Metadata Columns:", meta.columns.tolist())
print(meta.head())



📁 bathroom → 5 files
Example files: ['minimalist', 'boho', 'modern', 'scandinavian', 'industrial']

📁 bedroom → 5 files
Example files: ['scandinavian', 'modern', 'industrial', 'boho', 'minimalist']

📁 kitchen → 5 files
Example files: ['minimalist', 'scandinavian', 'modern', 'boho', 'industrial']

📁 living_room → 5 files
Example files: ['modern', 'scandinavian', 'minimalist', 'industrial', 'boho']

🧾 Metadata Columns: ['image_path', 'room_type', 'style']
                                        image_path room_type style
0    ../data/raw\bathroom\boho\bathroom_boho_0.jpg  bathroom  boho
1    ../data/raw\bathroom\boho\bathroom_boho_1.jpg  bathroom  boho
2   ../data/raw\bathroom\boho\bathroom_boho_10.jpg  bathroom  boho
3  ../data/raw\bathroom\boho\bathroom_boho_100.jpg  bathroom  boho
4  ../data/raw\bathroom\boho\bathroom_boho_101.jpg  bathroom  boho


In [6]:
import os
import pandas as pd
import glob

base_dir = "/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata"
meta_path = os.path.join(base_dir, "metadata.csv")

meta = pd.read_csv(meta_path)
fixed_rows = []

for _, row in meta.iterrows():
    room = str(row.get("room_type", "")).strip().lower().replace(" ", "_")
    style = str(row.get("style", "unknown")).strip().lower()
    filename = str(row.get("filename", "")).strip()

    # Try to find the image anywhere under base_dir
    found_files = glob.glob(os.path.join(base_dir, "**", filename), recursive=True)

    if found_files:
        fixed_rows.append({
            "image_path": found_files[0],
            "room_type": room,
            "style": style,
            "caption": f"A {style} style {room} interior design."
        })
    else:
        print("⚠️ Missing:", filename)

clean_df = pd.DataFrame(fixed_rows)
out_path = "/content/interior_captions_fixed.csv"
clean_df.to_csv(out_path, index=False)
print(f"\n✅ Fixed dataset saved to: {out_path}")
print(f"Total matched images: {len(clean_df)}")



✅ Fixed dataset saved to: /content/interior_captions_fixed.csv
Total matched images: 4139


In [7]:
import pandas as pd
df = pd.read_csv("/content/interior_captions_fixed.csv")
print(df.sample(5))
print("✅ Total valid images:", len(df))


                                             image_path    room_type  \
881   /content/drive/MyDrive/archive (11)/Pinterest ...      bedroom   
3129  /content/drive/MyDrive/archive (11)/Pinterest ...  living_room   
1778  /content/drive/MyDrive/archive (11)/Pinterest ...      kitchen   
1743  /content/drive/MyDrive/archive (11)/Pinterest ...      kitchen   
478   /content/drive/MyDrive/archive (11)/Pinterest ...     bathroom   

           style                                      caption  
881   industrial  A industrial style bedroom interior design.  
3129        boho    A boho style living_room interior design.  
1778        boho        A boho style kitchen interior design.  
1743        boho        A boho style kitchen interior design.  
478       modern     A modern style bathroom interior design.  
✅ Total valid images: 4139


In [8]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to("cuda")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [9]:
from diffusers import DiffusionPipeline
from diffusers.utils import load_image
from peft import LoraConfig
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from torch.utils.data import Dataset
from datasets import load_dataset

# Prepare dataset
# Added missing parenthesis
dataset = load_dataset("csv", data_files="/content/interior_captions_fixed.csv")["train"]

# Load model components
model = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet")
lora_config = LoraConfig(r=8, lora_alpha=32, target_modules=["to_q", "to_v"])
model.add_adapter(lora_config)

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

image = Image.open("/content/drive/MyDrive/archive (11)/Pinterest Interior Design Images and Metadata/living_room/industrial/living_room_industrial_1.jpg").convert("RGB")
prompt = "A minimalist style living room interior design."
image_out = pipe(prompt=prompt, image=image, strength=0.8, guidance_scale=7.5).images[0]

image_out.save("/content/stylized_output.jpg")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]